<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
<br>汉化的库: <a href="https://github.com/GoatCsu/CN-LLMs-from-scratch.git">https://github.com/GoatCsu/CN-LLMs-from-scratch.git</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 主数据加载流程概述

完整的章节代码位于[ch02.ipynb](./ch02.ipynb)。

该笔记本包含了数据加载流程的主要要点，但不包括中间步骤。

本章所需的包如下所示

In [1]:
# NBVAL_SKIP
# 仅用于打印依赖版本，方便复现；不影响后续数据加载与嵌入示例
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.10.0
tiktoken version: 0.12.0


In [2]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

# 这个笔记本展示一个最小可运行的数据加载流程：
# 1) 原始文本 -> token id 序列（整数列表）
# 2) token id 序列 -> (input_ids, target_ids) 训练样本（滑动窗口）
# 3) DataLoader 按 batch 产出张量，便于送入模型
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        # 保存每个样本的输入与目标：
        # - input_ids: 长度为 max_length 的上下文 token id
        # - target_ids: input_ids 向右平移 1 位（next-token prediction 的监督信号）
        self.input_ids = []
        self.target_ids = []

        # 把整段文本一次性编码成 token id 序列（Python list[int]）
        # allowed_special 允许编码结果中包含特殊 token（例如 <|endoftext|>）
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # 用滑动窗口把 token_ids 切成许多训练样本
        # - max_length: 每个样本的长度（上下文长度）
        # - stride: 窗口每次向右移动多少个 token；stride < max_length 会产生重叠样本
        # 例：token_ids=[t0,t1,t2,t3,t4,...], max_length=4
        # - i=0: input=[t0,t1,t2,t3], target=[t1,t2,t3,t4]
        # - i=1: input=[t1,t2,t3,t4], target=[t2,t3,t4,t5]
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]

            # 把 list[int] 转成 tensor；DataLoader 会在 batch 维度上自动堆叠
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        # 数据集样本数 = 可以滑动出的窗口数
        return len(self.input_ids)

    def __getitem__(self, idx):
        # 返回第 idx 个样本：(input_ids, target_ids)
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size, max_length, stride,
                         shuffle=True, drop_last=True, num_workers=0):
    # 创建 DataLoader 的便捷函数：把分词器、Dataset、DataLoader 封装起来
    # - shuffle: 是否打乱样本顺序（训练通常 True；演示/复现可 False）
    # - drop_last: 最后一个 batch 不满 batch_size 时是否丢弃
    # - num_workers: 并行加载进程数（本章示例用 0 便于跨平台）

    # 初始化 GPT-2 的 BPE 分词器（tiktoken 提供）
    tokenizer = tiktoken.get_encoding("gpt2")

    # 构造 Dataset：内部会把 raw text 切成一对对 (input_ids, target_ids)
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 构造 DataLoader：每次迭代返回一个 batch
    # - x 的 shape: (batch_size, max_length)
    # - y 的 shape: (batch_size, max_length)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader


# 读取示例文本（本章用它来构造训练样本）
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# GPT-2 (BPE) 词表大小；Embedding 的输出维度通常等于模型的隐藏维度
vocab_size = 50257
output_dim = 256

# 位置嵌入表能覆盖的最大位置数（绝对位置：0,1,2,...,context_length-1）
context_length = 1024


# token embedding: token id -> 向量，权重矩阵形状 (vocab_size, output_dim)
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
# position embedding: position id -> 向量，权重矩阵形状 (context_length, output_dim)
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

batch_size = 8
# 为了方便观察张量形状，这里用很短的 max_length=4（真实训练会更长）
max_length = 4
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=batch_size,
    max_length=max_length,
    stride=max_length  # stride=max_length 表示相邻样本不重叠
)

In [3]:
# 从 DataLoader 取出一个 batch（这里只演示第 1 个 batch）
for batch in dataloader:
    # x: 输入 token id，y: 目标 token id（均为整型张量）
    # x.shape == y.shape == (batch_size, max_length)
    x, y = batch

    # token_embedding_layer 会把每个 token id 查表变成 output_dim 维向量
    # token_embeddings.shape: (batch_size, max_length, output_dim)
    token_embeddings = token_embedding_layer(x)

    # 绝对位置嵌入：先生成位置 id [0, 1, ..., max_length-1]
    # torch.arange(max_length).shape: (max_length,)
    # pos_embeddings.shape: (max_length, output_dim)
    pos_embeddings = pos_embedding_layer(torch.arange(max_length))

    # 相加时会发生广播：pos_embeddings 会被“复制”到每个 batch 上
    # input_embeddings.shape: (batch_size, max_length, output_dim)
    input_embeddings = token_embeddings + pos_embeddings

    break

In [4]:
# 打印最终输入嵌入的张量形状：应为 (batch_size, max_length, output_dim)
print(input_embeddings.shape)

torch.Size([8, 4, 256])
